In [1]:
import torch
from datasets import load_dataset

ds = load_dataset("stanfordnlp/imdb")

C:\Users\Utilisateur\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
from datasets import load_from_disk

ds_local = load_from_disk("data/imdb_dataset")
ds_local

C:\Users\Utilisateur\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [62]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [2]:
class CustomImdbDataset:
    def __init__(self, dataset):
        self.reviews = dataset['text']
        self.labels = dataset['label']
        
    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, idx):
        label = self.labels[idx]
        review = self.reviews[idx]
        
        return review, label

In [3]:
train_ds = CustomImdbDataset(ds['train'])
len(train_ds)

25000

In [4]:
import re
text = "Just testing </br>"
# text = re.sub(r'<.*?>', '', text)
from nltk.tokenize import word_tokenize
word_tokenize(text)

['Just', 'testing', '<', '/br', '>']

In [60]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
tokenizer

BertTokenizerFast(name_or_path='bert-base-cased', vocab_size=28996, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [5]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
class TextProcessing:
    def __init__(self, lemmatizer=WordNetLemmatizer(), stop_words=None):
        self.lemmatizer = lemmatizer
        self.stop_words = stop_words

    def __call__(self, text):
        text = re.sub(r'<.*?>', '', text)
        text = re.sub(r'[^a-zA-Z\s]', '', text)
        text = text.lower()

        tokens = word_tokenize(text)
        
        if self.stop_words is not None:
            tokens = [token for token in tokens if not token in self.stop_words]
        
        lemmatized_tokens = [self.lemmatizer.lemmatize(token) for token in tokens]
        
        return lemmatized_tokens     

In [6]:
tp = TextProcessing(stop_words=None)
tp(train_ds[0][0])[:5]

['i', 'rented', 'i', 'am', 'curiousyellow']

In [7]:
stop_words = set(("the", "a", "this", "in", "of", "by", "and", "but", "or", "i", "it", "you", "just", "maybe", "there"))
tp = TextProcessing(stop_words=stop_words)
tp(train_ds[0][0])[:5]

['rented', 'am', 'curiousyellow', 'from', 'my']